# RTS Data Preprocessing

In [1]:
import json
import tokenize
import numpy as np
import pandas as pd
import os


BASE_PATH ='data/RTS_dataset_per_year/{}/{}'
parties = ['UDC', 'PDC', 'PS', 'PLR', 'PES', 'PVL']
years = np.arange(2012, 2020)

In [2]:
#Next, we want to go year by year and find interviews that are with interviews of political parties
    
def read_in(party):
    party_dict =[]
    
    for year in years:
        if os.path.exists(BASE_PATH.format(year, party)):
            files = [f for f in os.listdir(BASE_PATH.format(year, party))]
            for file in files:
                with open(BASE_PATH.format(year, party)+"/{}".format(file), 'r') as fp:
                    dicts = json.load(fp)
                    for entry in dicts['data']:
                        if 'genres' in entry:
                            if 'Interview' in entry['genres']:
                                party_dict.append(dicts['data'])
    return party_dict

#For each party, convert the json into dictionaries

UDC_dictionary = read_in('UDC')
PDC_dictionary = read_in('PDC')
PS_dictionary = read_in('PS')
PLR_dictionary = read_in('PLR')
PES_dictionary = read_in('PES')
PVL_dictionary = read_in('PVL')

Above, the json created by querying the RTS archive API for each of the party abbreviations was created into lists of dictionaries. However, it was noted that RTS does not often use the abbreviations for the PES and PVL parties and prefers to use the Verts and Verts libéreaux. Another query was made using the key word "verts" to find more broadcasts for these parties. Below these broadcasts were sorted into either verts or verts libéraux

In [3]:
#Below looks and sees if libéraux is in the title of an entry, if it is then the entry is sorted to the PVL dictionary

for year in years:
    if os.path.exists(BASE_PATH.format(year, 'verts')):
            files = [f for f in os.listdir(BASE_PATH.format(year, 'verts'))]
            for file in files:
                with open(BASE_PATH.format(year, 'verts')+"/{}".format(file), 'r') as fp:
                    dicts = json.load(fp)
                    for entry in dicts['data']:
                        if ('title' and 'genres') in entry:
                            if 'Interview' in entry['genres']:
                                if 'libéraux' in entry['title']:
                                    PVL_dictionary.append(entry)
                                else:
                                    PES_dictionary.append(entry)


After reading in the lists of dictionaries, each dictionary is converted to a pandas dataframe. During on our analysis we will look at either the "speechToTexts" or "summary" collumns in order to do some NLP to determine what the key issues each party were discussing and potentially their sentiment about these topics.

In [4]:
UDC_pandas = pd.DataFrame(UDC_dictionary)
PDC_pandas = pd.DataFrame(PDC_dictionary)
PS_pandas = pd.DataFrame(PS_dictionary)
PLR_pandas = pd.DataFrame(PLR_dictionary)
PES_pandas = pd.DataFrame(PES_dictionary)
PVL_pandas = pd.DataFrame(PVL_dictionary)